In [2]:
"""Takes several pre-baked cover letters in txt form, replaces placeholder text, and saves as a docx file"""

from docx import Document #pip install python-docx
from docx.shared import Pt, Inches
import os.path

def covlet_selector(POS,COMP,ADDR,TYPE):
    
    replace_dict = {'{POS}':POS,'{COMP}':COMP,'{ADDR}':ADDR}
    
    if TYPE.lower() == 'data':
        with open('CovLetTemplates/data.txt', encoding='utf-8') as f:
            content = f.read()
    elif TYPE.lower() == 'econ':
        with open('CovLetTemplates/econ.txt', encoding='utf-8') as f:
            content = f.read()
    elif TYPE.lower() == 'ml':
        with open('CovLetTemplates/ml.txt', encoding='utf-8') as f:
            content = f.read()
    elif TYPE.lower() == 'cons':
        with open('CovLetTemplates/cons.txt', encoding='utf-8') as f:
            content = f.read()
    elif TYPE.lower() == 'gis':
        with open('CovLetTemplates/gis.txt', encoding='utf-8') as f:
            content = f.read()
    
    for placeholder, actual in replace_dict.items():
        content = content.replace(placeholder, actual)
    return content
    
def take_first_letters(string):
    firstlet = []
    for s in string.upper().split():
        if not s.startswith('('): #Parenthesis not informative
            if s not in ['I','II','III', 'IV', 'V']: #These are numerals, not words. Do not abbreviate
                firstlet.append(s[0])
            else:
                firstlet.append('-'+s) #e.g. Business Analyst-III
        else:
            firstlet.append(s[1]) #If starts with open parenthesis, take letter after it.
    return ''.join(firstlet)

def create(POS,COMP,ADDR,TYPE):
    #Create document object
    doc = Document()
    
    #parse cover letter content
    content = covlet_selector(POS,COMP,ADDR,TYPE)
    
    #write to it
    texts = content.splitlines()
    for text in texts:
        para = doc.add_paragraph().add_run(text) ##This is counted as ONE paragraph, so para spacing don't work
        style = doc.styles['Normal']
        ###Font
        style.font.name = 'Times New Roman'
        style.font.size = Pt(12)
        ###Spacing
        para_format = style.paragraph_format
        para_format.space_before = Pt(4)
        para_format.space_after = Pt(8)
        para_format.line_spacing = 1.08
        ###Margins
        sections = doc.sections
        for section in sections:
            section.top_margin = Inches(1)
            section.bottom_margin = Inches(1)
            section.left_margin = Inches(1)
            section.right_margin = Inches(1)

    #create save folder, if not exist            
    if not os.path.exists('CovLet'):
        os.makedirs('CovLet')
        
    #savefile path: file naming convention should be informative to all who see it.

    forbidden = ["*", '"', "/", "\\", """<""", """>""", ":", """|"""] #forbidden for windows
    remove_forbidden = str.maketrans('', '', ''.join(forbidden)) #remove forbidden characters so Windows can save the file
    
    company = COMP.translate(remove_forbidden)
    position = POS.translate(remove_forbidden)
    
    path = F'CovLet/Tandaw_{company}_{position}.docx'
    
    #Short full-length filenames can be descriptive
    if len(path[6:])<45: 
        if os.path.isfile(path.replace(" ",""))==False:
            doc.save(path.replace(" ","")) #we replace here because we need whitespace for the else block to work
        
    #if file too long, use first letters only
    else:
        if len(company)>11: #long company names should be abbreviated
            shortpath = F"CovLet/Tandaw_{take_first_letters(company)}_{take_first_letters(position)}.docx"
            doc.save(shortpath)
        else: #short company names do not need to be abbreviated
            less_shortpath = F"CovLet/Tandaw_{company}_{take_first_letters(position)}.docx"
            doc.save(less_shortpath)

    return F"{TYPE.capitalize()} cover letter to {COMP} for the position of {POS} has been created in directory/CovLet folder"

In [3]:
create("Test Position",
       "Test Company",
       "Test City",
       "econ")

'Econ cover letter to International Food Policy Research Institute for the position of Research Analyst I/II has been created in directory/CovLet folder'